In [13]:
import os
import re
import numpy as np
import pandas as pd
import jieba
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix

from bs4 import BeautifulSoup

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import load_model

In [14]:
def load_dataset(name, nrows=None):
    datasets = {
        'labeled_train': 'train_first.csv',
        'test': 'predict_first.csv'
    }
    if name not in datasets:
        raise ValueError(name)
    data_file = os.path.join('..', 'ccf_data', datasets[name])
    df = pd.read_csv(data_file, sep=',', nrows=nrows)
    print('Number of reviews: {}'.format(len(df)))
    return df

## 2.把训练集和测试集的评论都放进来训练word2vec

In [15]:
df1 = load_dataset('labeled_train').Discuss
df2 = load_dataset('test').Discuss
df_unlabeled = df1.append(df2)
df_unlabeled.index = range(len(df_unlabeled)) # 重新整理下index
print(df_unlabeled.shape)

Number of reviews: 100000
Number of reviews: 30000
(130000,)


In [4]:
def clean_text(text, remove_stopwords=False):
    text = re.sub(r'good|nice|excellent|beautiful','不错',text)
    words = list(jieba.cut(text))
    words = [re.sub(r'[^0-9\u4E00-\u9FA5]+','',s1).replace(' ','') for s1 in words]
    words = [s2 for s2 in words if s2]
    if remove_stopwords:
        words = [w for w in words if w not in chn_stopwords]
    return words
def split_sentences(review):
    raw_sentences = re.split(r'。|！|？|～',review.lower().strip())
    sentences = [clean_text(s,remove_stopwords=False) for s in raw_sentences if s]
    return sentences

In [5]:
%%time
sentences = sum(df_unlabeled.apply(split_sentences), [])
print('{} reviews -> {} sentences'.format(len(df_unlabeled), len(sentences)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\16121360\AppData\Local\Temp\jieba.cache
Loading model cost 1.030 seconds.
Prefix dict has been built succesfully.


130000 reviews -> 222159 sentences
Wall time: 5min 13s


## 3.用gensim训练词嵌入模型

In [95]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# 设定词向量训练的参数
num_features = 200    # Word vector dimensionality
min_word_count = 5   # Minimum word count  这里把词频低的删掉了所以tokenizer会比gensim生成的word2vec多
num_workers = 4       # Number of threads to run in parallel
context = 5          # Context window size
downsampling = 1e-3   # Downsample setting for frequent words

model_name = '{}features_{}minwords_{}context.model'.format(num_features, min_word_count, context)

In [96]:
print('Training model...')
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model.save(os.path.join('..', 'ccf_models', model_name))

2018-02-27 17:17:29,858 : INFO : collecting all words and their counts
2018-02-27 17:17:29,859 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-27 17:17:29,893 : INFO : PROGRESS: at sentence #10000, processed 138088 words, keeping 19230 word types
2018-02-27 17:17:29,937 : INFO : PROGRESS: at sentence #20000, processed 280151 words, keeping 29237 word types
2018-02-27 17:17:29,970 : INFO : PROGRESS: at sentence #30000, processed 420333 words, keeping 36450 word types
2018-02-27 17:17:30,006 : INFO : PROGRESS: at sentence #40000, processed 561384 words, keeping 42384 word types
2018-02-27 17:17:30,044 : INFO : PROGRESS: at sentence #50000, processed 698417 words, keeping 47284 word types
2018-02-27 17:17:30,078 : INFO : PROGRESS: at sentence #60000, processed 839308 words, keeping 52223 word types


Training model...


2018-02-27 17:17:30,116 : INFO : PROGRESS: at sentence #70000, processed 980361 words, keeping 56568 word types
2018-02-27 17:17:30,148 : INFO : PROGRESS: at sentence #80000, processed 1117003 words, keeping 60373 word types
2018-02-27 17:17:30,186 : INFO : PROGRESS: at sentence #90000, processed 1255707 words, keeping 63997 word types
2018-02-27 17:17:30,220 : INFO : PROGRESS: at sentence #100000, processed 1398259 words, keeping 67400 word types
2018-02-27 17:17:30,259 : INFO : PROGRESS: at sentence #110000, processed 1540123 words, keeping 70681 word types
2018-02-27 17:17:30,293 : INFO : PROGRESS: at sentence #120000, processed 1684669 words, keeping 73853 word types
2018-02-27 17:17:30,334 : INFO : PROGRESS: at sentence #130000, processed 1823218 words, keeping 76422 word types
2018-02-27 17:17:30,372 : INFO : PROGRESS: at sentence #140000, processed 1963001 words, keeping 78818 word types
2018-02-27 17:17:30,410 : INFO : PROGRESS: at sentence #150000, processed 2104062 words, kee

In [5]:
# 测试下similarity
model.similar_by_word('不错')

D:\programs\Anaconda3.5\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  from ipykernel import kernelapp as app


[('挺不错', 0.752413272857666),
 ('很棒', 0.7250882983207703),
 ('好', 0.6435768008232117),
 ('挺好玩', 0.6095079183578491),
 ('很赞', 0.5767607688903809),
 ('很漂亮', 0.5721205472946167),
 ('滴', 0.5358286499977112),
 ('总的来说', 0.5355515480041504),
 ('出外景', 0.5351769924163818),
 ('一般般', 0.5249240398406982)]

## 读入之前训练好的Word2Vec模型 或者用别人训练好的word2vec模型.model or .vector

In [16]:
## .model文件
model_name = '200features_5minwords_5context.model'
model = Word2Vec.load(os.path.join('..', 'ccf_models', model_name))

## .bin文件
# model_name = 'zh.bin'
# model = Word2Vec.load_word2vec_format(os.path.join('..', 'ccf_models', model_name),binary=True)

## .vector文件
# model_name = 'wiki.zh.text.vector'
# model = gensim.models.KeyedVectors.load_word2vec_format(os.path.join('..', 'ccf_models', model_name),binary=False)

## 4.用RNN构建分类器 [这里用的是Keras LSTM模型]

In [17]:
MAX_SEQUENCE_LENGTH = 560  # 557
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 200  # 词向量维度
VALIDATION_SPLIT = 0.25
word_index = 22932 # 训练和测试合起来不同词的数量
hidden_layers_num = 100 # 这里训练集评论有10w所有100够用了，如果数据量再大点，可以选更大的隐藏层数量

In [18]:
# 这个model是之前用gensim包生成的word2vec(词频小于5的都过滤掉了)
embedding_matrix = model.wv.vectors
embedding_matrix.shape

(28539, 200)

### 将gensim生成的word2vec变成字典的形式 比如{'不错':[0.02,0.1,....0.09]}

In [34]:
word_vec = {}
for word in model.wv.vocab:
    word_vec[word] = model[word]

C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


### 数据准备

In [35]:
texts = []
labels = []

In [36]:
def split_clean_text2(text, remove_stopwords=False):
    text = re.sub(r'。|！|？|～',' ',text.lower().strip())
    text = re.sub(r'good|nice|excellent|beautiful','不错',text)
#     text = BeautifulSoup(text, 'html.parser').get_text()
    words = list(jieba.cut(text))
    words = [re.sub(r'[^0-9\u4E00-\u9FA5]+','',s1).replace(' ','') for s1 in words]
    words = [s2 for s2 in words if s2]
    if remove_stopwords:
        words = [w for w in words if w not in chn_stopwords]
    return ' '.join(words)

In [37]:
# texts和labels准备 texts包含测试数据转索引使用
# 这里的texts包含了训练和测试数据，每句话转成索引以后，再将训练和测试切分开
texts = df_unlabeled.apply(split_clean_text2)
df1_score = load_dataset('labeled_train').Score
df1_score = df1_score - 1 # tricky
labels = keras.utils.to_categorical(df1_score)

Number of reviews: 100000


In [38]:
# 将每个不重复的单词生成一个索引
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
del texts
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

C:\Program Files\Anaconda3\lib\site-packages\keras\preprocessing\text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 97623 unique tokens.


In [39]:
# 将每句话索引进行填充，使得每句话的维度一样
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [40]:
# 训练和测试转成索引以后进行分割
x_train_data = data[:100000]
x_test = data[100000:]
del data

In [29]:
### 将训练数据分成训练集和验证集0.75:0.25
indices = np.arange(x_train_data.shape[0])
np.random.shuffle(indices)
x_train_data = x_train_data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * x_train_data.shape[0])

x_train = x_train_data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = x_train_data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

### 将embedding根据tokenize的索引进行重排序

In [ ]:
# 根据维度生成一个新的embedding_matrix并且初始化
embedding_matrix2 = np.random.random((len(word_index) + 1, EMBEDDING_DIM))

In [32]:
# 根据每个不同词的索引将embedding_matrix的行插入到embedding_matrix2中，没有的就保留原来生成embedding_matrix2时候的随机数
# 注：这里embedding_matrxi的行和embedding_matrix2中的行有差异是因为，用gensim生成embedding_matrix的时候低频词被过滤掉了只有2.8w个
# 而进行tokenize的时候所有词都保留有9w多个不同的词
for word, i in word_index.items():
    embedding_vector = word_vec.get(word)
    if embedding_vector is not None:
        # words not found in word_vec will be random number.
        embedding_matrix2[i] = embedding_vector

In [33]:
# keras model compile我们发现非常的简单
embedding_layer = Embedding(len(word_index)+1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix2],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
del embedding_matrix2
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
preds = Dense(5, activation='softmax')(l_lstm)
keras_model = Model(sequence_input, preds)
keras_model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [39]:
# 模型拟合(很费时间，建议用GPU跑)
keras_model.fit(x_train,y_train,validation_data=(x_val,y_val),batch_size=5000,epochs=10)

Train on 75000 samples, validate on 25000 samples
Epoch 1/10
75000/75000 [==============================] - 1891s 25ms/step - loss: 1.0237 - acc: 0.5426 - val_loss: 0.9484 - val_acc: 0.5979
Epoch 2/10
75000/75000 [==============================] - 1887s 25ms/step - loss: 0.9354 - acc: 0.6054 - val_loss: 0.9280 - val_acc: 0.5990
Epoch 3/10
75000/75000 [==============================] - 1875s 25ms/step - loss: 0.9045 - acc: 0.6122 - val_loss: 0.8990 - val_acc: 0.6126
Epoch 4/10
75000/75000 [==============================] - 1877s 25ms/step - loss: 0.8524 - acc: 0.6381 - val_loss: 0.8642 - val_acc: 0.6270
Epoch 5/10
75000/75000 [==============================] - 1878s 25ms/step - loss: 0.8251 - acc: 0.6514 - val_loss: 0.8573 - val_acc: 0.6289
Epoch 6/10
75000/75000 [==============================] - 1872s 25ms/step - loss: 0.8010 - acc: 0.6656 - val_loss: 0.8656 - val_acc: 0.6329
Epoch 7/10
75000/75000 [==============================] - 1877s 25ms/step - loss: 0.7717 - acc: 0.6796 - val_l

In [49]:
## save model
keras_model_name = 'keras_model_xval.h5'
keras_model.save(os.path.join('..', 'ccf_models', keras_model_name))

In [ ]:
## load model
from keras.models import load_model
keras_model_name = 'keras_model_xval.h5'
keras_model = load_model(os.path.join('..', 'ccf_models', keras_model_name))

In [55]:
%%time
a = keras_model.predict(x_test,batch_size=5000)

Wall time: 3min 56s


In [58]:
a

array([[4.69519902e-04, 7.80403148e-04, 6.79166988e-03, 8.44874755e-02,
        9.07470942e-01],
       [5.91006537e-04, 8.44240480e-04, 1.68418027e-02, 1.94144696e-01,
        7.87578285e-01],
       [1.59064017e-03, 1.99473323e-03, 4.63031307e-02, 1.20853715e-01,
        8.29257727e-01],
       ...,
       [5.02938638e-03, 6.87745120e-03, 9.66919586e-02, 2.39155918e-01,
        6.52245343e-01],
       [1.46296353e-03, 1.75840966e-03, 2.82078814e-02, 1.13249727e-01,
        8.55321050e-01],
       [3.81560321e-03, 6.93328958e-03, 1.02820881e-01, 7.40468740e-01,
        1.45961553e-01]], dtype=float32)

In [96]:
c = pd.DataFrame(a)
d = c.apply(np.argmax,axis=1)

In [97]:
d.value_counts()

4    21016
3     6637
2     2347
dtype: int64

In [84]:
df_tmp = load_dataset('test')

Number of reviews: 30000


In [93]:
output = pd.DataFrame({'Id':df_tmp.Id,'Score':d})

In [94]:
output.to_csv(os.path.join('..', 'ccf_data', 'submit.csv'),sep=',',index=False,header=False)